In [145]:
%load_ext autoreload
%autoreload 2
import os
from collections import defaultdict
if os.path.basename(os.path.realpath(os.getcwd())) == 'notebooks':
    print("changing dir...")
    os.chdir('../')
print(os.getcwd())

import glob
import os

class Text:
    def __init__(self, idx, raw_text):
        self.raw_text = raw_text
        self.index = idx
        self.labels_1 = []
        self.labels_2 = []


class Label:
    def __init__(self, passage_idx,  question, label, annotator=1):
        self.passage_idx = passage_idx
        self.label = label
        self.question = question
        self.annotator = annotator
    def __eq__(self, other):
        return (self.passage_idx == other.passage_idx and
                self.label == other.label and
                self.question == other.question)

    def __repr__(self):
        return f"{self.question}: {self.label}"

labels = [
    'Altering',
    'Nonaltering',
    'Ungrammatical',
    'QuestionAnswer',
    'BadExample',
    'Opposite'
    ]
def load(file, split=True):
    with open(file) as f:
        d = f.read()
        if split:
            d = d.splitlines()
        return d
def read_aligned_annotation(path, aligned = True):
    collection = []
    for f in sorted(glob.glob(os.path.join(path, "*coannotate*","*.txt"))):
        print(f)
        aligned_f = f.replace('-coannotate', "")
        ann_f = f.replace('.txt', '.ann')
        raw_text = load(f, split=False)
        raw_text_aligned = load(aligned_f, split=False)
        assert raw_text == raw_text_aligned
        aligned_ann_f = aligned_f.replace('.txt', '.ann')
        text_idx = int(f.split('.', 1)[0].rsplit('/', 1)[-1])
        t = Text(text_idx, raw_text)
        for l in load(ann_f):
            if l.startswith("T"):
                idx, label_type, start_index, end_index, surface_form = l.replace("\t", " ").split(" ", 4)
                start_index, end_index = int(start_index), int(end_index)
                if label_type in labels:
                    question_text = raw_text[end_index:raw_text[end_index+1:].index('\n')+end_index+1].strip()
                    label = Label(text_idx, question_text, label_type, 1)
                    if label not in t.labels_1:
                        t.labels_1.append(label)

        t.labels_1 = sorted(t.labels_1, key=lambda l: l.question)
        for l in load(aligned_ann_f):
            if l.startswith("T"):
                idx, label_type, start_index, end_index, surface_form = l.replace("\t", " ").split(" ", 4)
                start_index, end_index = int(start_index), int(end_index)
                if label_type in labels:
                    question_text = raw_text_aligned[end_index:raw_text_aligned[end_index+1:].index('\n')+end_index+1].strip()
                    label = Label(text_idx, question_text, label_type, 1)
                    if label not in t.labels_2:
                        assert any(l.question == question_text for l in t.labels_1), f"{text_idx} {t.labels_1}, {label}"
                        t.labels_2.append(label)
        assert len(t.labels_1) == len(t.labels_2), text_idx
        t.labels_2 = sorted(t.labels_2, key=lambda l: l.question)
        collection.append(t)
    return collection

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/viktor/workspace/mrc-stresstest


In [146]:
collection = read_aligned_annotation('data/brat-data-annotated/')

data/brat-data-annotated/drop-coannotate/007.txt
data/brat-data-annotated/drop-coannotate/017.txt
data/brat-data-annotated/drop-coannotate/021.txt
data/brat-data-annotated/drop-coannotate/027.txt
data/brat-data-annotated/drop-coannotate/031.txt
data/brat-data-annotated/drop-coannotate/038.txt
data/brat-data-annotated/drop-coannotate/052.txt
data/brat-data-annotated/drop-coannotate/061.txt
data/brat-data-annotated/drop-coannotate/065.txt
data/brat-data-annotated/drop-coannotate/068.txt
data/brat-data-annotated/drop-coannotate/069.txt
data/brat-data-annotated/drop-coannotate/073.txt
data/brat-data-annotated/drop-coannotate/078.txt
data/brat-data-annotated/drop-coannotate/079.txt
data/brat-data-annotated/drop-coannotate/081.txt
data/brat-data-annotated/drop-coannotate/083.txt
data/brat-data-annotated/drop-coannotate/085.txt
data/brat-data-annotated/drop-coannotate/088.txt
data/brat-data-annotated/drop-coannotate/091.txt
data/brat-data-annotated/drop-coannotate/095.txt
data/brat-data-annot

In [154]:
label_map = {l: i for i, l in enumerate(labels)}
for l, i in label_map.items():
    print(l,i)

Altering 0
Nonaltering 1
Ungrammatical 2
QuestionAnswer 3
BadExample 4
Opposite 5


In [155]:
ann_1 = []
ann_2 = []
for text in collection:
    for l1, l2 in zip(text.labels_1, text.labels_2):
        assert l1.question == l2.question
        ann_1.append(label_map[l1.label])
        ann_2.append(label_map[l2.label])

from sklearn.metrics import cohen_kappa_score
print(cohen_kappa_score(ann_1, ann_2,labels = [1,2,3,4,5]))

0.8225084988370013


In [165]:
def read_annotation(path):
    collection = []
    for f in sorted(glob.glob(os.path.join(path, "*","*.txt"))):
        if 'coannotate' not in f:
            ann_f = f.replace('.txt', '.ann')
            raw_text = load(f, split=False)
            text_idx = int(f.split('.', 1)[0].rsplit('/', 1)[-1])
            t = Text(text_idx, raw_text)
            for l in load(ann_f):
                if l.startswith("T"):
                    idx, label_type, start_index, end_index, surface_form = l.replace("\t", " ").split(" ", 4)
                    start_index, end_index = int(start_index), int(end_index)
                    if label_type in labels:
                        question_text = raw_text[end_index:raw_text[end_index+1:].index('\n')+end_index+1].strip()
                        label = Label(text_idx, question_text, label_type, 1)
                        if label not in t.labels_1:
                            t.labels_1.append(label)

            t.labels_1 = sorted(t.labels_1, key=lambda l: l.question)
            collection.append(t)
    return collection

In [167]:
single_annotation = read_annotation('data/brat-data-annotated/')
print(len(single_annotation))

400


In [170]:
from collections import Counter
c = Counter()
total = 0
for t in single_annotation:
    for l in t.labels_1:
        c[l.label] += 1
        total += 1
print(c)

Counter({'Nonaltering': 522, 'QuestionAnswer': 61, 'BadExample': 36, 'Opposite': 27, 'Altering': 1})


In [180]:
import tabulate
print(total)
print(tabulate.tabulate([[l,v, f"{v / total:.2%}"] for l, v in c.items()]))

647
--------------  ---  ------
Nonaltering     522  80.68%
QuestionAnswer   61  9.43%
Altering          1  0.15%
BadExample       36  5.56%
Opposite         27  4.17%
--------------  ---  ------
